### <p style="text-align: center;">Practical 5</p>

#### <p style="text-align: center;">Implementation of C4.5 Decision Tree Classifier</p>

In [1]:
import numpy as np
import pandas as pd
import pprint

In [2]:
df = pd.read_excel("datasets\Lab 4 ML Dataset - ID3.xlsx")

##### Entropy Function for whole DataFrame


In [3]:
def find_entropy(df):
    Class = df.keys()[-1]
    values = df[Class].unique()
    entropy = 0
    for value in values:
        prob = df[Class].value_counts()[value]/len(df[Class])
        entropy += -prob * np.log2(prob)
    return np.float64(entropy)

##### Entropy Function for specific Attribute

In [4]:
def find_entropy_attribute(df, attribute):
    Class = df.keys()[-1]
    target_values = df[Class].unique()
    attribute_values = df[attribute].unique()
    avg_entropy = 0
    for value in attribute_values:
        entropy = 0
        for value1 in target_values:
            num = len(df[attribute][df[attribute] == value][df[Class] == value1])
            den = len(df[attribute][df[attribute] == value])
            prob = num/den
            entropy += -prob * np.log2(prob + 0.000001)
        avg_entropy += (den/len(df))*entropy
    return np.float64(avg_entropy)

##### Split Info

In [6]:
def splitInfo(table,attribute):
    counts = table[attribute].value_counts()
    split_info = 0
    for i in counts:
        split_info = split_info + (i/ len(table)) * np.log2(i/ len(table))
    return -1*(split_info + 0.00000001)

##### Find Winner (Finding Node with maximum IG)

In [7]:
def find_winner(df):
    GR = []
    for key in df.keys()[:-1]:
        gain = find_entropy(df) - find_entropy_attribute(df, key)
        GR.append(gain/splitInfo(df,key))
    return df.keys()[:-1][np.argmax(GR)]

In [8]:
def get_subtable(df, attribute, value):
    return df[df[attribute] == value].reset_index(drop = True)

##### Function To build The Tree

In [9]:
def buildtree(df, tree = None):
    node = find_winner(df) # Find Root Node
    attvalue = np.unique(df[node]) #Find uinque values of the Current Node
    Class = df.keys()[-1]
    if tree is None:
        tree = {}
        tree[node] = {}
    for value in attvalue:
        subtable = get_subtable(df,node,value)
        Clvalue, counts = np.unique(subtable[Class], return_counts = True)
        if len(counts) == 1:
            tree[node][value] = Clvalue[0]
        else:
            tree[node][value] = buildtree(subtable)
    return tree

##### Function To make Predictions


In [10]:
def predict(inst, tree):
    for node in tree.keys():
        value = inst[node]
        tree = tree[node][value]
        prediction = 0
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
    return prediction 

#### Building Tree

In [11]:
tree = buildtree(df)

In [12]:
pprint.pprint(tree)

{'age': {'Midage': 'yes',
         'senior': {'credit_rating': {'excellent': 'no', 'fair': 'yes'}},
         'young': {'student': {'no': 'no', 'yes': 'yes'}}}}


#### Making Prediction

In [13]:
sample = [["young","high","yes","excellent"],
          ["Midage","low","no","excellent"]]
df1 = pd.DataFrame(sample,columns = ["age","income","student","credit_rating"])

In [14]:
df1

,age,income,student,credit_rating
0,young,high,yes,excellent
1,Midage,low,no,excellent


In [15]:
Y_label = []
for i in range(len(df1)):
    inst = df1.iloc[i,:]
    prediction = predict(inst, tree)
    Y_label.append(prediction)
print(Y_label)

['yes', 'yes']
